# Reinforcement Learning

Reinforcement learning is a kind of machine learning that relies on trial and error to improve.
> Note: Generally, the agents are tested in 'batches' or generations, and the agent(s) with the highest 'fitness' is(are) selected and mutated or changed slightly in the next generation. This allows for iterative improvement. However, this is not an example of this. Here, only one generation is run and the best from that generation is selected. This means that instead of each generation improving on the last, all the agents are in direct competition with each other and the best one is random. 
>
> Additionally, the agents are often run in parallel each episode but only one agent is run per episode here for simplicity and understandability.

There are two main categories of AI that I will address here. There are neural networks which is the AI we hear most about as they are designed to mimic organic/human brains, and there is deterministic AI or in this specific case, policy iteration. Since the principles are similar between the two, here, deterministic ai is used to more easily show the fundamentals of machine learning to take inputs and generate outputs based on that

## Neural Networks

Neural networks are designed to digitally simulate the behaviour of real neurons in organic brains. In organic brains, each neuron has various connections or inputs and when enough of them are activated the neuron is activated and sends a signal to the next neuron in the chain which can lead to emergent complexity. In the digital case, the neurons are broken up into layers. The first layer of neurons, also called the input layer, will take inputs from its environment as defined by you such as the current velocity of the car. Each neuron in the input layer will then be connected to some number of neurons in the second layer (the first 'hidden' layer). Each of these connections with the second layer has some weight which, when multiplied by the input value will give some output that is given to the neuron in the second layer. Each neuron in the second layer will also have some bias assigned to it that will be added to the sum value from all of its input connections. Then, as the neuron can only have a value between 0 and 1, the output will be compressed or normalised between these values so that ever smaller negative numbers asymptotically approach 0 and ever larger numbers will asymptotically approach 1 and values between 0 and 1 will remain far less compressed in the mid ranges. This 'compression' is usually done with the sigmoid function. Each neuron in the second layer will also have some number of connections with neurons in the second layer, each with random weights etc. until the final hidden layer connects to the output layers where a value between 0 and 1 will be generated and can be used for whatever you want such as to control the throttle of the car.

In reinforcement learning, many of these neural networks will be generated with different weights, different connections and different hidden neurons (if using NEAT), and different biases on each neuron. Then each network will be tested in a training environment and given a fitness score. This is based on how well the AI completes the assigned task. This may be how far right the car is. Then the neural network(s) judged to have the highest fitness will be preserved in the next generation and have copies made of them with slight changes or mutations to the weights, connections and hidden neurons (if using NEAT), and biases. This will give the agents slightly different behaviour which may be judged as better or worse by the fitness algorithm and the process repeats until the AI can complete the task to the required quality.

## Deterministic Policy AI

Deterministic policy ai is much simpler than neural networks but functions on the same basic principles of all ai (take some input and make a given output) and reinforcement learning (iterate on the best of each generation). The main difference being that policy ai uses a simple(r) 'lookup table' to decide the output for a given input. This means that every input defined in the observation space has a corresponding output in the action space. These values for the lookup table are generated randomly as with all reinforcement learning models and the best is then selected and can be iterated on. 

While policy AI leads to less complex behavior and an inability to respond to new environments, they do serve as an effective way to perform machine learning when used correctly and serve as a good and simple example as shown below.
## Example Walkthrough of Reinforcement Learning

### Importing libraries

First, the 'gym' library needs to be imported. If you don't have this library installed, run `pip install gym` in terminal. 

In [49]:
import gym; # This is the library that we will use to create the neural networks and an environment to test them in.

### Creating the Environment

Using the gym library, we then get one of the pre-made environments. I used the [Mounatin Car](https://www.gymlibrary.dev/environments/classic_control/mountain_car/) Environment as seen below but many more can be found at the [gym website](https://www.gymlibrary.dev/environments/classic_control/).

![Mountain Car Evironment](../assets/images/MountainCar.gif)

### About the Environment

In this environment the goal for the car is to reach the flag at the top of the hill on the right. This is equivilent to the car x position being larger than or equal to 0.5. In each timestep the ai can either accelerate left (by outputting 0), do nothing (by outputting 1), or accelerate right (by outputting 2) depending on where on the hill it is. It knows where on the hill it is through its 'observation space' which are its x position and its velocity. By accelerating left when it is travelling left and right when it is travelling right the ai can reach the goal in the least time (perfect time calculated to be between 10 and 100 actions).

In [50]:
env = gym.make("MountainCar-v0"); # Creates the Mountain Car Environment v0 (there is only one version excluding the continuous mountain car variation).

### Setting and Tuning AI Parameters

Here the episodes or the number of trials and the number of actions each ai can take per trial. Increasing the episodes will increase how many trials there are and how refined/good the AI are. This will increase the proficiency of the AI, however, it will also increase the amount of time it will take to run the AI. Increasing the number of actions will also increase the time it takes for the AI to train, however, it will not increase the proficiency of the AI, just the amount of time the AI has to complete the task. You should often aim to keep both as low as possible to reduce training time and in many cases to avoid overfitting but increasing it will make the AI better at completing the training example(s). You should also try to make sure that episodes is high enough that the best AI can get the task done within the actions limit otherwise you will not know or be able to see if the AI is improving or learning properly and the AI won't be able to complete the task. After some testing, I found that 10000 and 1000 for the episodes and actions respectively optimised the training time while also effectively guaranteeing that the AI would be able to complete the time in the actions limit.

In [51]:
episodes = 10000; # Sets the number of 'episodes', trials, or agents.
actions = 1000; # Sets the number of actions the agents are allowed to take.

### Running the AI

Now that we have the environment set up and the parameters set we can actually run the AI reinforcement learning algorithm. Here we get each ai to perform a random series of actions and measure how long each AI takes to reach the goal (if it ever reaches the goal). The time the AI takes to reach the goal is the proxy for its fitness and we then save this AI and play it back later to see what actions it takes and make sure the code worked to train the AI.

In [52]:
best = actions; # Records the lowest number of actions. Set to the actions limit and reduced anytime an ai reaches the flag in less time.
bestActions = []; # Saves the actions of the best ai whenever the best variable is changed.

for a in range(episodes): # Repeats for each new trial/episode to test each agent.
    initial_state=env.reset(); # Reset the environment to start the trial on the latest ai and gets the input ready for the first action of the agent.
    ranActions = []; # Saves the actions of this ai incase it is the best ai.

    for b in range(best): # Repeats for each action the ai takes that is less than the best so far. This stops wasting time on ai that are not going to be better than the best ai.
        random_action = env.action_space.sample() # Gets a random action from the set of possible actions.
        observation, reward, terminated, truncated, info = env.step(random_action); # Perform the random action and store all the outputs incase you want to use them.
        ranActions.append(random_action); # This adds the last action of the ai to the ranActions array which saves the actions of this ai in case it is the best ai. \

        if(terminated): # If the ai completes the level before it runs out of actions then exit and check if it is the best so far.
            if(best > b): # If the ai is the best so far, save its progress and print its stats.
                best = b; # Save the ai's progress.
                bestActions = ranActions; # This saves the actions of the best ai so that the actions/ai can be played back later.
                print("Least steps:", best, "  At episode", a); # Print the ai's stats.
            
            env.close(); # End the trial.
            break; # Exit the action loop and run the next ai in the episode loop.

for b in range(len(bestActions)): # Repeats for each action the best ai takes.
    observation, reward, terminated, truncated, info = env.step(bestActions[b]); # Perform the action that the best ai took and store all the outputs to print them.
    print("step", b, bestActions[b], observation, reward, terminated, info); # Print all the outputs from the last action.

print("Least number of steps: ", best); # Print the final best score again at the end to signal that it is done and has reached this peak performance.

Least steps: 864   At episode 574
Least steps: 817   At episode 5827
Least steps: 693   At episode 6375
Least steps: 690   At episode 9446
Least steps: 622   At episode 9662
step 0 2 [-0.56293035 -0.01711221] -1.0 False {}
step 1 2 [-0.5787483  -0.01581791] -1.0 False {}
step 2 1 [-0.5941545  -0.01540617] -1.0 False {}
step 3 2 [-0.6080354  -0.01388095] -1.0 False {}
step 4 0 [-0.62228984 -0.01425442] -1.0 False {}
step 5 2 [-0.6348148  -0.01252501] -1.0 False {}
step 6 1 [-0.6465211  -0.01170628] -1.0 False {}
step 7 2 [-0.65632623 -0.00980511] -1.0 False {}
step 8 2 [-0.664162   -0.00783575] -1.0 False {}
step 9 0 [-0.6719745 -0.0078125] -1.0 False {}
step 10 0 [-0.6797105  -0.00773606] -1.0 False {}
step 11 1 [-0.6863181  -0.00660756] -1.0 False {}
step 12 1 [-0.69175315 -0.00543506] -1.0 False {}
step 13 0 [-0.6969799  -0.00522672] -1.0 False {}
step 14 2 [-0.69996405 -0.00298421] -1.0 False {}
step 15 2 [-0.7006864  -0.00072232] -1.0 False {}
step 16 2 [-0.69914216  0.00154423] -1